# Solving SAT With Grover
In this notebook, we aim to solve satisfyability formulae using grover search.
For this, we first construct marking oracles for the basic gates of AND, OR and NOT.
Then, we transform them into the required phase-oracle-form.

The main resources used are this [Microsoft Kata](https://github.com/microsoft/QuantumKatas/blob/ec925ccfcb599a6bb29c8a39035d0b149f447f9a/SolveSATWithGrover/Workbook_SolveSATWithGrover.ipynb) and this [Qiskit Textbook Chapter](https://qiskit.org/textbook/ch-applications/satisfiability-grover.html).

## Marking Oracles
### And

In [ ]:
from qiskit.quantum_info.operators import Operator

## Define Basic Matrices
I = Operator([[1,0],[0,1]])
X = Operator([[0,1],[1,0]])

def sized_identity(nqubits):
    """Creates an identity operator of given size"""
    unitary = I
    for i in range(nqubits-1):
        unitary = unitary.tensor(I)
    return unitary

def marking_and_oracle(nInputRegister):
    """Returns a matrix representation of the and oracle"""
    nqubits = nInputRegister+1;
    unitary = sized_identity(nqubits)
    
    # Apply the x gate to the output register
    unitary.data[-1][-1] = 0
    unitary.data[-2][-2] = 0
    unitary.data[-1][-2] = 1
    unitary.data[-2][-1] = 1
    return unitary 

### Or

In [ ]:
def sized_x(nqubits):
    """Creates a not operator of given size"""
    unitary = X
    for i in range(nqubits-1):
        unitary = unitary.tensor(X)
    return unitary

def marking_or_oracle(nInputRegisters):
    """Returns a matrix representation of the or oracle"""
    nqubits = nInputRegisters+1
    unitary = sized_x(nqubits).dot(marking_and_oracle(nInputRegisters))
    unitary = unitary.dot(sized_x(nInputRegisters).tensor(I))
    return unitary

While we could go on further and create _XOR_ and _AlternatingBits_ oracles but we don't need them for now so let's not :)

## SAT Oracles
Using our marking gate-oracles, we can create oracles for evaluating SAT clauses and finally complete functions.

### Single Clause Oracle
A Clause is a disjunction of variables (qubits) that are potentially negated.
The clause `x0 || !x1` may be represented by the input `[(0, true), (1, false)]`.

In [ ]:
def marking_clause_oracle(qubits: [int], target: int, clause: [(int,bool)]):
    nqubits = len(qubits) + target
    unitary = sized_identity(nqubits)
    # Flip input qubits
    for index, positive in clause:
        if not positive:
            unitary = unitary.
            